<a href="https://colab.research.google.com/github/gitmystuff/AgenticAI/blob/main/11_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangGraph

* https://langchain-ai.github.io/langgraph/
* https://langchain-ai.github.io/langgraph/#langgraphs-ecosystem
* LangGraph
* LangGraph Studio
* LangGraph Platform


## Agentic Design Patterns

* Anthropic Building Effective Agents - https://www.anthropic.com/engineering/building-effective-agents  

## Knowledge Graphs

### Core Concepts of Knowledge Graphs

* **Nodes**: Represent individual entities (people, places, things) and are the fundamental units of information.
* **Edges** (or **Relationships**): Connect nodes and define the associations between them (e.g., "works for," "is located in").
* **Triples**: The basic building block, forming a subject-predicate-object statement by connecting two nodes with an edge.
* **Entities**: The individual things or concepts represented in the graph.
* **Relationships**: Describe how entities are connected.
* **Ontologies**: Provide a schema or structure for the graph, defining the types of entities and relationships that exist.
* **Inference**: The process of using rules to deduce new facts based on existing knowledge within the graph.
* **Context**: Provided by linking data together with semantic information, aiding in understanding the meaning and relationships.


### Related Terms and Technologies

* **Semantic Data Modeling**: The methodology used to represent the meaning and relationships between data points, making them machine-understandable.
* **Data Ingestion**: The process of gathering and integrating data from various sources into the knowledge graph.
* **Resource Description Framework (RDF)**: A standard model for representing information in knowledge graphs using subject-predicate-object triples.
* **SPARQL**: A specific query language designed for retrieving and manipulating data from RDF graphs.
* **Graph Theory**: The mathematical framework that knowledge graphs draw from for understanding and analyzing networks of interconnected nodes.
* **Knowledge Base**: A general term; knowledge graphs are a type of knowledge base containing structured information for reasoning and decision-making.

These concepts collectively allow knowledge graphs to represent complex relationships and support applications like search and AI systems.


### Knowledge Base

### Graph Databases

### LangGraph Terminology

https://langchain-ai.github.io/langgraph/concepts/low_level/

* State
* Nodes
* Edges

### LangGraph Process

* Define the State class
* Start the Graph Builder
* Create a Node
* Create Edges
* Compile the Graph

## Getting Started

In [ ]:
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
import random


In [ ]:
load_dotenv(override=True)

In [ ]:
# Some useful constants

nouns = ["Cabbages", "Unicorns", "Toasters", "Penguins", "Bananas", "Zombies", "Rainbows", "Eels", "Pickles", "Muffins"]
adjectives = ["outrageous", "smelly", "pedantic", "existential", "moody", "sparkly", "untrustworthy", "sarcastic", "squishy", "haunted"]

### State Object

* Dict (TypedDict)
* Pydantic BaseModel

### Type Hinting

### LangGraph's Annotated

In [ ]:
def shout(text: Annotated[str, "Something to shout about"]) -> str:
  print(text.upper())
  return text.upper()

### Reducer

* Hadoop's Map-Reduce?

### 1. Define the State Object with Pydantic BaseModel

In [ ]:
class State(BaseModel):
  messages: Annotated[list, add_messages]

### 2. Start the Graph Builder

In [ ]:
graph_builder = StateGraph(State)

### 3. Create a Node

In [ ]:
def our_first_node(old_state: State) => State:
  reply = f"{random.choice(nouns)} are {random.choice(adjectives)}"
  messages = [{"role": "assistant", "content": reply}]
  new_state = State(messages=messsages)
  return new_state

graph_builder.add_node("first_node", our_first_node)

### 4. Create Edges

In [ ]:
graph_builder.add_edge(START, "first_node")
graph_builder.add_edge("first_node", END)

### 5. Compile the Graph

In [ ]:
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaild_png()))

## LLM Implementation

In [ ]:
def chat(user_input: str, history):
    message = {"role": "user", "content": user_input}
    messages = [message]
    state = State(messages=messages)
    result = graph.invoke(state)
    print(result)
    return result["messages"][-1].content

gr.ChatInterface(chat, type="messages").launch()

In [ ]:
# Step 1: Define the State object
class State(BaseModel):
    messages: Annotated[list, add_messages]

In [ ]:
# Step 2: Start the Graph Builder with this State class
graph_builder = StateGraph(State)

In [ ]:
# Step 3: Create a Node
llm = ChatOpenAI(model="gpt-4o-mini")

def chatbot_node(old_state: State) -> State:
    response = llm.invoke(old_state.messages)
    new_state = State(messages=[response])
    return new_state

graph_builder.add_node("chatbot", chatbot_node)

In [ ]:
# Step 4: Create Edges
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

In [ ]:
# Step 5: Compile the Graph
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
def chat(user_input: str, history):
    initial_state = State(messages=[{"role": "user", "content": user_input}])
    result = graph.invoke(initial_state)
    print(result)
    return result['messages'][-1].content

gr.ChatInterface(chat, type="messages").launch()

## Memory

In [ ]:
from typing import Annotated
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain_openai import ChatOpenAI
from typing import TypedDict

load_dotenv(override=True)

### Super Step

A super-step in LangGraph is a single, discrete, and transactional unit of execution within a graph. All active nodes within a super-step execute concurrently, and their combined state updates are applied atomically, meaning either all updates succeed or none do, ensuring state consistency.

A super-step can be considered a single iteration over the graph nodes. Nodes that run in parallel are part of the same super-step, while nodes that run sequentially belong to separate super-steps.

### LangSmith

LangSmith is a unified platform developed by LangChain for **debugging, testing, evaluating, and monitoring** applications built with large language models (LLMs).

Essentially, it provides the necessary tools to take your LLM prototypes (often built with frameworks like LangChain or LangGraph) and make them production-ready and reliable.

Key aspects of LangSmith include:

* **Observability (Tracing):** It allows you to visualize and understand the step-by-step execution of your LLM applications, including prompts, responses, tool calls, and intermediate steps. This is crucial for debugging the non-deterministic nature of LLMs.
* **Evaluation:** LangSmith helps you assess the performance and quality of your LLM applications. You can create datasets, run experiments, and use various evaluators (including LLM-as-a-Judge or custom ones) to score application performance and gather human feedback.
* **Prompt Engineering:** It provides tools like a Playground to iterate on prompts, compare different versions, and collaborate with your team to find the most effective prompts.
* **Monitoring:** You can track key business metrics like costs, latency, and response quality in live dashboards, and set up alerts for issues in production.

In short, while frameworks like LangChain help you **build** LLM applications, LangSmith helps you **ensure they work reliably and perform well** in real-world scenarios.

### Tools

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper

serper = GoogleSerperAPIWrapper()
serper.run("What is the capital of France?")

### Using LangChain Wrapper

In [ ]:
from langchain.agents import Tool

tool_search =Tool(
        name="search",
        func=serper.run,
        description="Useful for when you need more information from an online search"
    )


### Custom Tool

In [ ]:
# pushover_token = os.getenv("PUSHOVER_TOKEN")
# pushover_user = os.getenv("PUSHOVER_USER")
# pushover_url = "https://api.pushover.net/1/messages.json"

def push(text: str):
    """Send a push notification to the user"""
    # requests.post(pushover_url, data = {"token": pushover_token, "user": pushover_user, "message": text})
    return text

tool_push = Tool(
        name="send_push_notification",
        func=push,
        description="useful for when you want to send a push notification"
    )

tool_push.invoke("Hello World!")

In [ ]:
tools = [tool_search, tool_push]

In [ ]:
# Step 1: Define the State object
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
# Step 2: Start the Graph Builder with this State class
graph_builder = StateGraph(State)

In [ ]:
# Step 3: Create a Node
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

In [ ]:
# Step 4: Create Edges
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")

# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

In [ ]:
# Step 5: Compile the Graph
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

### Interface

In [ ]:
def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]})
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

### Memory

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

# Steps 1 and 2
graph_builder = StateGraph(State)

# Step 3
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    print(state)
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

# Step 4
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# Step 5
graph = graph_builder.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "1"}}

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

In [ ]:
graph.get_state(config)

In [ ]:
# Most recent first
list(graph.get_state_history(config))

In [ ]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

db_path = "memory.db"
conn = sqlite3.connect(db_path, check_same_thread=False)
sql_memory = SqliteSaver(conn)

# Steps 1 and 2
graph_builder = StateGraph(State)

# Step 3
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    print(state)
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

# Step 4
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# Step 5
graph = graph_builder.compile(checkpointer=sql_memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "3"}}

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content

gr.ChatInterface(chat, type="messages").launch()